In [1]:
import requests
import shelve
import pandas as pd

In [22]:
params = {
    'url': 'http://localhost:3000/customer-information',
    'params': {'document': '09089356000118'}
}
resp = requests.get(**params).json()
resp

{'status_code': 200,
 'message': 'Success',
 'data': {'informacoes_cadastrais': {'abertura_conta': '2022-08-12',
   'categoria_conta': 'grande',
   'classficacao_conta': 'E'},
  'informacoes_transacionais': {'media_transacoes_tres_meses': 2712.44,
   'quantidade_transacoes': 47,
   'saldo': 64061.48,
   'faturamento': 971786},
  'informacoes_bureau_credito': {'score_risco': 376,
   'pendencia_financeira': 0,
   'anotacoes_negativas': 0},
  'informacoes_receita_federal': {'cnpj': '09089356000118',
   'razao_social': 'GERENCIANET S.A. - INSTITUICAO DE PAGAMENTO',
   'natureza_juridica': 'Sociedade Anônima Fechada',
   'qualificacao_responsavel': 'Diretor',
   'capital_social': 31301696.0,
   'porte': '',
   'ente_federativo_responsavel': None,
   'nome_fantasia': None,
   'situacao_cadastral': 'Ativa',
   'data_situacao_cadastral': '2007-09-27',
   'motivo_situacao_cadastral': 'SEM MOTIVO',
   'nome_da_cidade_no_exterior': None,
   'pais': None,
   'endereco': {'tipo_logradouro': 'AVENID

In [2]:
from tinydb import TinyDB, Query

# Abrir (ou criar) um banco de dados TinyDB
db = TinyDB('files/database/kvstore_tinydb.json')
KeyValue = Query()

In [119]:
db.get(KeyValue.analyses_id == 1).get('data')

{'status_code': 200,
 'message': 'Success',
 'data': {'informacoes_cadastrais': {'abertura_conta': '2022-08-12',
   'categoria_conta': 'grande',
   'classficacao_conta': 'E'},
  'informacoes_transacionais': {'media_transacoes_tres_meses': 2712.44,
   'quantidade_transacoes': 47,
   'saldo': 64061.48,
   'faturamento': 971786},
  'informacoes_bureau_credito': {'score_risco': 376,
   'pendencia_financeira': 0,
   'anotacoes_negativas': 0},
  'informacoes_receita_federal': {'cnpj': '09089356000118',
   'razao_social': 'GERENCIANET S.A. - INSTITUICAO DE PAGAMENTO',
   'natureza_juridica': 'Sociedade Anônima Fechada',
   'qualificacao_responsavel': 'Diretor',
   'capital_social': 31301696.0,
   'porte': '',
   'ente_federativo_responsavel': None,
   'nome_fantasia': None,
   'situacao_cadastral': 'Ativa',
   'data_situacao_cadastral': '2007-09-27',
   'motivo_situacao_cadastral': 'SEM MOTIVO',
   'nome_da_cidade_no_exterior': None,
   'pais': None,
   'endereco': {'tipo_logradouro': 'AVENID

In [37]:
data = {}
for key in db.get(KeyValue.analyses_id == 1).get('data').get('data').keys():
    data.update(db.get(KeyValue.analyses_id == 1).get('data').get('data').get(key))
data

{'abertura_conta': '2022-08-12',
 'categoria_conta': 'grande',
 'classficacao_conta': 'E',
 'media_transacoes_tres_meses': 2712.44,
 'quantidade_transacoes': 47,
 'saldo': 64061.48,
 'faturamento': 971786,
 'score_risco': 376,
 'pendencia_financeira': 0,
 'anotacoes_negativas': 0,
 'cnpj': '09089356000118',
 'razao_social': 'GERENCIANET S.A. - INSTITUICAO DE PAGAMENTO',
 'natureza_juridica': 'Sociedade Anônima Fechada',
 'qualificacao_responsavel': 'Diretor',
 'capital_social': 31301696.0,
 'porte': '',
 'ente_federativo_responsavel': None,
 'nome_fantasia': None,
 'situacao_cadastral': 'Ativa',
 'data_situacao_cadastral': '2007-09-27',
 'motivo_situacao_cadastral': 'SEM MOTIVO',
 'nome_da_cidade_no_exterior': None,
 'pais': None,
 'endereco': {'tipo_logradouro': 'AVENIDA',
  'logradouro': 'PRESIDENTE JUSCELINO KUBITSCHEK',
  'numero': '31',
  'complemento': 'CONJ  A                   PAVMTO2 E 3',
  'bairro': 'BAUXITA',
  'cep': '35402179',
  'uf': 'MG',
  'municipio': 'OURO PRETO'},


In [39]:
def key_exists_any_level(data, key):
    if isinstance(data, list):
        return any(key_exists_any_level(item, key) for item in data)
    elif isinstance(data, dict):
        if key in data:
            return True
        return any(key_exists_any_level(value, key) for value in data.values())
    return False

# Função para verificar se a chave existe em qualquer nível do banco
def key_exists_in_db(key):
    return any(key_exists_any_level(entry, key) for entry in db.all())

exists = key_exists_in_db('classficacao_conta')
exists

True

In [14]:
rules = [
    {
        'rule': 'categoria_conta == "grande"',
        'code': 'PEN001',
        'descricao': 'Categoria da conta Grande',
        'action': 'pendency'
    },
    {
        'rule': "classficacao_conta in ['Fraude']",
        'code': 'REP001',
        'descricao': 'Categoria da conta Grande',
        'action': 'reproved'
    },
    # {
    #     'rule': 'bloqueio == True',
    #     'code': 'REP002',
    #     'descricao': 'Bloqueio na conta',
    #     'action': 'reproved'
    # }    
]

In [41]:
layer_result = 'ERRO'
final_result = 'APROVADO'
priority = ['ERRO', 'REPROVADO', 'PENDENTE', 'APROVADO']
priority[min([priority.index(layer_result), priority.index(final_result)])]

'ERRO'

In [77]:
def handle_final_result(layer_result, final_result):
    priority = ['ERRO', 'REPROVADO', 'PENDENTE', 'APROVADO']
    return priority[min([priority.index(layer_result), priority.index(final_result)])]

def decisao(rules, layer):
    decisao_rule_rep = []
    decisao_rule_pen = []
    errors = []
    for rule in rules:
        try:
            pd.eval(rule.get('rule'), local_dict=data)
            result = eval(rule.get('rule'), {}, data)
            if result:
                variavel = set(var for var in data.keys() if var in rule.get('rule')).pop()
                rule['variavel_regra'] = {variavel: data.get(variavel)}
                if rule.get('action') == 'refuse':
                    decisao_rule_rep.append(rule)
                else:
                    decisao_rule_pen.append(rule)
        except Exception as err:
            decisao_rule_error = rule.copy()
            decisao_rule_error['error_detail'] = str(err)
            errors.append(decisao_rule_error)
    layer_result = 'PENDENTE' if len(decisao_rule_pen) > 0 else 'APROVADO'
    layer_result = 'REPROVADO' if len(decisao_rule_rep) > 0 else layer_result
    layer_result = 'ERRO' if len(errors) > 0 else layer_result
    
    decisao = {
        'processo': layer,
        'decisao': layer_result,
        'codigos_rep': decisao_rule_rep if decisao_rule_rep else None,
        'codigos_pen': decisao_rule_pen if decisao_rule_pen else None,
    }
    if errors:
        decisao['erros'] = errors
    return decisao

In [73]:
df_policy_rule = pd.read_clipboard(sep=',')
df_policy_rule

,policy_id,policy_name,layer_id,layer_name,rule_id,action,rule_name,code,rule,description
0,2,PoliticaCartaoPF,2,P1-CamadaBaixoCusto,1,refuse,REP001-FaturamentoPJ,REP020,faturamento == 0,Regra para faturamento
1,2,PoliticaCartaoPF,2,P1-CamadaBaixoCusto,3,refuse,REP003-SituacaoDocumento,REP003,situacao_cadastral != 'Ativa',Situação do documento na receita
2,2,PoliticaCartaoPF,2,P1-CamadaBaixoCusto,4,pendency,PEN001-ContaGrande,PEN001,categoria_conta == 'grande',Conta grande
3,2,PoliticaCartaoPF,3,P2-CamadaMedioCusto,2,refuse,REP002-ScoreSerasa,REP002,score_risco > 800,Regra para score Serasa


In [78]:
response = []
grupos = df_policy_rule.groupby('layer_name')

final_result = 'APROVADO'
for grupo, dados_grupo in grupos:
    rules = dados_grupo[['rule_name', 'code', 'rule', 'description', 'action']].to_dict('records')
    decision_layer = decisao(rules, grupo)
    response.append(decision_layer)
    final_result = handle_final_result(decision_layer.get('decisao'), final_result)

final_decision = {
    'decision': final_result,
    'decision_details': response
}
print(json.dumps(final_decision, indent=4, ensure_ascii=False))

{
    "decision": "PENDENTE",
    "decision_details": [
        {
            "processo": "P1-CamadaBaixoCusto",
            "decisao": "PENDENTE",
            "codigos_rep": null,
            "codigos_pen": [
                {
                    "rule_name": "PEN001-ContaGrande",
                    "code": "PEN001",
                    "rule": "categoria_conta == 'grande'",
                    "description": "Conta grande",
                    "action": "pendency",
                    "variavel_regra": {
                        "categoria_conta": "grande"
                    }
                }
            ]
        },
        {
            "processo": "P2-CamadaMedioCusto",
            "decisao": "APROVADO",
            "codigos_rep": null,
            "codigos_pen": null
        }
    ]
}


In [80]:
import numpy as np
from collections import OrderedDict

In [83]:
classification_matrix = OrderedDict([('E', 0),('D', 0),('C', 0.4),('B', 0.6),('A', 0.8),('AA', 1)])
classification_matrix.keys()
classification_matrix.values()

odict_values([0, 0, 0.4, 0.6, 0.8, 1])

In [90]:
score = np.arange(500)
score

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [115]:
def matrix(self, score: int):
    conditions = [
        (score >= 0 and score < 100),
        (score >= 100 and score < 200),
        (score >= 200 and score < 400),
        (score >= 400 and score < 600),
        (score >= 600 and score < 800),
        (score >= 800 and score < 1000)
    ]
    classification_matrix = OrderedDict([('E', 0),('D', 0),('C', 0.4),('B', 0.6),('A', 0.8),('AA', 1)])
    x = np.arange(6)
    classification = np.select(conditions, classification_matrix.keys(), default='erro')
    perc_limit = np.select(conditions, classification_matrix.values(), default=0)
    return classification.item(), perc_limit.item()

In [116]:
x, y = matrix('', 500)

In [118]:
x, y

('B', 0.6)

In [108]:
type(x)

numpy.ndarray

In [106]:
dir(x)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__o

In [32]:
db.insert({'item': 'request_data', 'analyses_id': 2, 'data': resp})

2

In [35]:
db.search(KeyValue.analyses_id == 2)

[{'item': 'request_data',
  'analyses_id': 2,
  'data': {'status_code': 200,
   'message': 'Success',
   'data': {'informacoes_cadastrais': {'abertura_conta': '2022-08-12',
     'categoria_conta': 'grande',
     'classficacao_conta': 'E'},
    'informacoes_transacionais': {'media_transacoes_tres_meses': 2712.44,
     'quantidade_transacoes': 47,
     'saldo': 64061.48,
     'faturamento': 971786},
    'informacoes_bureau_credito': {'score_risco': 376,
     'pendencia_financeira': 0,
     'anotacoes_negativas': 0},
    'informacoes_receita_federal': {'cnpj': '09089356000118',
     'razao_social': 'GERENCIANET S.A. - INSTITUICAO DE PAGAMENTO',
     'natureza_juridica': 'Sociedade Anônima Fechada',
     'qualificacao_responsavel': 'Diretor',
     'capital_social': 31301696.0,
     'porte': '',
     'ente_federativo_responsavel': None,
     'nome_fantasia': None,
     'situacao_cadastral': 'Ativa',
     'data_situacao_cadastral': '2007-09-27',
     'motivo_situacao_cadastral': 'SEM MOTIVO'

In [30]:
db.all()

[{'item': 'request_data',
  'analyses_id': 1,
  'data': {'status_code': 200,
   'message': 'Success',
   'data': {'informacoes_cadastrais': {'abertura_conta': '2022-08-12',
     'categoria_conta': 'grande',
     'classficacao_conta': 'E'},
    'informacoes_transacionais': {'media_transacoes_tres_meses': 2712.44,
     'quantidade_transacoes': 47,
     'saldo': 64061.48,
     'faturamento': 971786},
    'informacoes_bureau_credito': {'score_risco': 376,
     'pendencia_financeira': 0,
     'anotacoes_negativas': 0},
    'informacoes_receita_federal': {'cnpj': '09089356000118',
     'razao_social': 'GERENCIANET S.A. - INSTITUICAO DE PAGAMENTO',
     'natureza_juridica': 'Sociedade Anônima Fechada',
     'qualificacao_responsavel': 'Diretor',
     'capital_social': 31301696.0,
     'porte': '',
     'ente_federativo_responsavel': None,
     'nome_fantasia': None,
     'situacao_cadastral': 'Ativa',
     'data_situacao_cadastral': '2007-09-27',
     'motivo_situacao_cadastral': 'SEM MOTIVO'

In [26]:


# Abrir (ou criar) um banco de dados shelve
with shelve.open('files/database/responses.db') as db:
    # db['item'] = 'request_data'
    # db['analyses_id'] = 2
    # db['data'] = resp
    dicionario = {key: value for key, value in db.items()}
dicionario

{'item': 'request_data',
 'analyses_id': 2,
 'data': {'status_code': 200,
  'message': 'Success',
  'data': {'informacoes_cadastrais': {'abertura_conta': '2022-08-12',
    'categoria_conta': 'grande',
    'classficacao_conta': 'E'},
   'informacoes_transacionais': {'media_transacoes_tres_meses': 2712.44,
    'quantidade_transacoes': 47,
    'saldo': 64061.48,
    'faturamento': 971786},
   'informacoes_bureau_credito': {'score_risco': 376,
    'pendencia_financeira': 0,
    'anotacoes_negativas': 0},
   'informacoes_receita_federal': {'cnpj': '09089356000118',
    'razao_social': 'GERENCIANET S.A. - INSTITUICAO DE PAGAMENTO',
    'natureza_juridica': 'Sociedade Anônima Fechada',
    'qualificacao_responsavel': 'Diretor',
    'capital_social': 31301696.0,
    'porte': '',
    'ente_federativo_responsavel': None,
    'nome_fantasia': None,
    'situacao_cadastral': 'Ativa',
    'data_situacao_cadastral': '2007-09-27',
    'motivo_situacao_cadastral': 'SEM MOTIVO',
    'nome_da_cidade_no_

In [18]:
db.keys()

KeysView(<shelve.DbfilenameShelf object at 0x000001B97D407B30>)

In [19]:
{key: value for key, value in db.items()}

ValueError: invalid operation on closed shelf